# Mapping around with Python and Leaflet

Leaflet is an amazing project used by R and Python and elswhere. It lets you create interactive maps with a number of basemap layers and other cool feature. Maps can then be embedded in web-apps or just used in your notbooks...or even screenshot-captured.

In this notebook we explore some of the basic functionality.

Tech info:

If you are on a windows machine and dont have pip installed, start cmd and run:

```curl https://bootstrap.pypa.io/get-pip.py | python```


Then run

```pip install ipywidgets ipyleaflet geopy```




In [132]:
# Leaflet controls so called widgets (webdev stuff) and we don't really have to understand that
# What you need to know: Load up these libraries
# Unfortunately documentation for ipyleaflet (it's a rather new library)

# Widgets
from ipywidgets import HTML
# Leaflet
from ipyleaflet import *

# and of cause pandas for data manipulation
import pandas as pd

In [133]:
# Let's start with a map of copenhagen
cph = Map(center=(55.677683, 12.569588), zoom=11)
#cph = Map(center=(55.677683, 12.569588), zoom=10, basemap=basemaps.Hydda.Full) # you can add other layers

In [134]:
cph

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [135]:
# Let's set a marker on our map
# we first define it

marker = Marker(location=(55.692913, 12.599274), draggable=False)

In [139]:
# And then place it on as a new layer
cph.add_layer(marker)

In [137]:
cph.remove_layer(marker)

In [140]:
# If you want, you can save the map as an html file and open in a browser (your data will be embedded)
from ipywidgets.embed import embed_minimal_html 

embed_minimal_html('map_sds.html',views=[cph])

Wouldn't it be nice to know what is in the spot where we placed the marker?
Sure thing:

![havefrue](http://a.bimg.dk/node-images/975/8/452x250-c/8975349-node-image-upload-den-lille-havfrue-til-debat-ks-foto-ren-eriksenjpg.jpg)

In [141]:
# We can create a "popup" where the value of the popup is a a bit of HTML code 
# I disect it below
# We then assign the popup to our marker

popup = HTML()
popup.value = "The Little Mermaid <img src='https://upload.wikimedia.org/wikipedia/en/thumb/7/7a/Copenhagen_-_the_little_mermaid_statue_-_2013.jpg/1024px-Copenhagen_-_the_little_mermaid_statue_-_2013.jpg' alt='Mermaid'>"
marker.popup = popup

HTML: Text, then image, then a short image placeholder text (no rocket science)

```HTML
"The Little Mermaid 
<img src='https://upload.wikimedia.org/wikipedia/en/thumb/7/7a/Copenhagen_-_the_little_mermaid_statue_-_2013.jpg/1024px-Copenhagen_-_the_little_mermaid_statue_-_2013.jpg' 
     alt='Mermaid'>"
```

Let's bring some more points on the map. This time public toilets in CPH you can get this data [here](https://data.kk.dk/dataset/toiletter)

In [142]:
# Now let's try to plot in some very important data

# import the json library to deal with json data (more on that later)
import json
data = json.load(open('cph_toilets.json'))

In [154]:
data['features'][0]['properties']

{'id': 168,
 'toilet_lokalitet': 'Nyhavn',
 'adresse': 'Nyhavn 1F, 1051 København K',
 'toilet_type': 'Opsyn',
 'toilet_betegnelse': 'Toilet med personale',
 'handicapadgang': 'Ja',
 'design': 'Underjordisk',
 'helaarsaabent': 'Ja',
 'aabent_april_sept': None,
 'mandag': '9.00 - 22.30',
 'tirsdag': '9.00 - 22.30',
 'onsdag': '9.00 - 22.30',
 'torsdag': '9.00 - 22.30',
 'fredag': '9.00 - 22.30',
 'loerdag': '9.00 - 22.30',
 'soendag': '9.00 - 22.30',
 'doegnaabent': 'Nej',
 'bemaerkning': 'Info om jul/nytår offentliggøres medio december hvert år.',
 'registreret_dato': '2002-11-12T23:00:00Z',
 'rettet_dato': '2018-07-24T08:41:34.517Z',
 'longitude': 12.587758,
 'latitude': 55.680573}

In [143]:
# Transform the data into a layer for our map
geo_json = GeoJSON(data=data)

In [144]:
# put it on
cph.add_layer(geo_json)

In [146]:
# and remove it again
cph.remove_layer(geo_json)

In [155]:
# We can do better :-)

# Let's load up the WC data as a dataframe
wc = pd.DataFrame([x['properties'] for x in data['features']])

```[x['properties'] for x in data['features']]``` is a list comprehension
Python has these amazingly short loop structures built in

Read:

for each element (here x) in the dictionary "data" under the key "fatures"
take the element and unpack the nested key "properties"
put all of these in a list

YES!!! We can pass a list of dictionaries to Pandas and it will still construct a neat dataframe


In [157]:
# Inspecting the comlumns
wc.columns

Index(['aabent_april_sept', 'adresse', 'bemaerkning', 'design', 'doegnaabent',
       'fredag', 'handicapadgang', 'helaarsaabent', 'id', 'latitude',
       'loerdag', 'longitude', 'mandag', 'onsdag', 'registreret_dato',
       'rettet_dato', 'soendag', 'tirsdag', 'toilet_betegnelse',
       'toilet_lokalitet', 'toilet_type', 'torsdag'],
      dtype='object')

Don't get scared: The next cell looks a bit tricky but it is not. We will now iterate over all rows of our dataframe and take out the location as well as create popup labels. All of that will be put into an empty list which we define in the very first row.
In the last step of the loop we are adding our circles to the map


In [158]:
circles = []
for x in wc.iterrows():
    loc = (x[1]['latitude'], x[1]['longitude'])
    message = HTML()
    message.value = x[1]['toilet_lokalitet'] + '<br />' + x[1]['adresse']
    c = Circle(location=loc, radius=3, color = "#4CB391", fill_opacity = 0.4, popup = message)
    circles.append(c)
    cph.add_layer(c)

In [ ]:
# We can also remove all of the by calling
for c in circles:
    cph.remove_layer(c)

## Geocoding

Geocoding is super important and means simply transforming address data into geocoordinates
Reverse geocoding is self explanatory

In [ ]:
# We will need to install and import geopy
# Geopy supports a large number of services (Google maps, Bing, Baidu etc.)
# We will use the Open Street Maps Nominatim (since that one doesn't require signing in and an API key)

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="SDS_geocode_teaching")

In [161]:
# I created a little dataset of pubs in Aalborg
bodega = pd.read_csv('bodega.txt', sep=';')

In [163]:
bodega

,bodega,address
0,Cafe Frederiksberg,"Hadsundvej 1B, 9000 Aalborg"
1,Den Lille Havfrue,"Hadsundvej 14, 9000 Aalborg"
2,Kahytten,"Hadsundvej 11A, 9000 Aalborg"
3,Vejgaard Kroen,"Hadsundvej 44, 9000 Aalborg"
4,Jægerstuen,"Vendsysselgade 2, 9000 Aalborg"
5,Søkroen,"Langelandsgade 2, 9000 Aalborg"
6,Færø-kroen/v Premovil A/S,"Færøgade 57, 9000 Aalborg"
7,Restaurant Østerport v/Patricia Nielsen,"Nørregade 32, 9000 Aalborg"
8,Hjerter Dame ApS,"Danmarksgade 96, 9000 Aalborg"


In [169]:
test_geo = geolocator.geocode('Fibigerstræde 4, aalborg')

In [176]:
# Not th emost elegant approach but it does work and is easy to read
# Simple iteration over the rows and geocoding

lonlist = []
latlist = []

for i in bodega.iterrows():
    loc = geolocator.geocode(i[1]['address'])
    lonlist.append(loc.longitude)
    latlist.append(loc.latitude)

In [179]:
# Enter the values

bodega['lat'] = latlist
bodega['lon'] = lonlist

In [180]:
bodega

,bodega,address,lat,lon
0,Cafe Frederiksberg,"Hadsundvej 1B, 9000 Aalborg",57.044281,9.949016
1,Den Lille Havfrue,"Hadsundvej 14, 9000 Aalborg",57.043119,9.949820
2,Kahytten,"Hadsundvej 11A, 9000 Aalborg",57.043472,9.949897
3,Vejgaard Kroen,"Hadsundvej 44, 9000 Aalborg",57.040748,9.951460
4,Jægerstuen,"Vendsysselgade 2, 9000 Aalborg",57.041704,9.942360
5,Søkroen,"Langelandsgade 2, 9000 Aalborg",57.042340,9.941162
6,Færø-kroen/v Premovil A/S,"Færøgade 57, 9000 Aalborg",57.041828,9.938773
7,Restaurant Østerport v/Patricia Nielsen,"Nørregade 32, 9000 Aalborg",57.046333,9.929052
8,Hjerter Dame ApS,"Danmarksgade 96, 9000 Aalborg",57.045363,9.928694


In [181]:
# A bit of a different map

aalborg = Map(center=(57.042340, 9.938773), zoom=14, basemap=basemaps.Stamen.Watercolor)

In [182]:
aalborg

Map(basemap={'url': 'http://stamen-tiles-a.a.ssl.fastly.net/watercolor/{z}/{x}/{y}.png', 'attribution': 'Map t…

In [183]:
# Let's do what we practiced above

circles = []

for x in bodega.iterrows():
    loc = (x[1]['lat'], x[1]['lon'])
    message = HTML()
    message.value = x[1]['bodega'] + '<br />' + x[1]['address']
    c = Circle(location=loc, radius=5, color = "#133366", fill_opacity = 0.6, popup = message)
    circles.append(c)
    aalborg.add_layer(c)

In [ ]:
# If you don't like the pubs they can go
for c in circles:
    aalborg.remove_layer(c)